# Thresholding

## Simple thresholding

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import skimage.color
import skimage.filters
%matplotlib widget

In [ ]:
%load_ext watermark
%watermark -v -p numpy,matplotlib,skimage
%matplotlib

In [ ]:
# load the image
image = skimage.io.imread("../../fig/06-junk-before.jpg")

fig, ax = plt.subplots()
plt.imshow(image)
plt.show()

In [ ]:
# convert the image to grayscale
gray_image = skimage.color.rgb2gray(image)

# blur the image to denoise
blurred_image = skimage.filters.gaussian(gray_image, sigma=1.0)

fig, ax = plt.subplots()
plt.imshow(blurred_image, cmap='gray')
plt.show()

In [ ]:
# create a histogram of the blurred grayscale image
histogram, bin_edges = np.histogram(blurred_image, bins=256, range=(0.0, 1.0))

fig, ax = plt.subplots()
plt.plot(bin_edges[0:-1], histogram)
plt.title("Grayscale Histogram")
plt.xlabel("grayscale value")
plt.ylabel("pixels")
plt.show()

In [ ]:
# create a mask based on the threshold
t = 0.8
binary_mask = blurred_image < t

fig, ax = plt.subplots()
plt.imshow(binary_mask, cmap='gray')
plt.show()

In [ ]:
# use the binary_mask to select the "interesting" part of the image
selection = np.zeros_like(image)
selection[binary_mask] = image[binary_mask]

fig, ax = plt.subplots()
plt.imshow(selection)
plt.show()

### More practice with simple thresholding

In [ ]:
image = skimage.io.imread("../../fig/06-more-junk.jpg")
gray_image = skimage.color.rgb2gray(image)
blurred_image = skimage.filters.gaussian(gray_image, sigma=1.0)

histogram, bin_edges = np.histogram(blurred_image, bins=256, range=(0.0, 1.0))

fig, ax = plt.subplots()
plt.plot(bin_edges[0:-1], histogram)
plt.title("Graylevel histogram")
plt.xlabel("gray value")
plt.ylabel("pixel count")
plt.xlim(0, 1.0)
plt.show()

In [ ]:
t = 0.5
binary_mask = blurred_image > t

fig, ax = plt.subplots()
plt.imshow(binary_mask, cmap='gray')
plt.show()

In [ ]:
selection = np.zeros_like(image)
selection[binary_mask] = image[binary_mask]

fig, ax = plt.subplots()
plt.imshow(selection)
plt.show()

## Adaptive thresholding

In [ ]:
image = skimage.io.imread("../../fig/06-roots-original.jpg")

# conver the image to grayscale
gray_image = skimage.color.rgb2gray(image)

# blur the image to denoise
blurred_image = skimage.filters.gaussian(gray_image, sigma=1.0)

# create the histogram
histogram, bin_edges = np.histogram(blurred_image, bins=256, range=(0.0, 1.0))

fig, ax = plt.subplots()
plt.plot(bin_edges[0:-1], histogram)
plt.title("Graylevel histogram")
plt.xlabel("gray value")
plt.ylabel("pixel count")
plt.xlim(0, 1.0)
plt.show()

In [ ]:
# perform adaptive thresholding
t = skimage.filters.threshold_otsu(blurred_image)
print("Found adaptive threshold t = {}.".format(t))

In [ ]:
# perform adaptive thresholding
binary_mask = blurred_image > t

fig, ax = plt.subplots()
plt.imshow(binary_mask, cmap='gray')
plt.show()

In [ ]:
# apply the binary mask to select the foreground
selection = np.zeros_like(image)
selection[binary_mask] = image[binary_mask]

fig, ax = plt.subplots()
plt.imshow(selection)
plt.show()

## Application: measuring root mass

In [ ]:
import glob
all_files = glob.glob("trial-???.jpg")

fig, ax = plt.subplots(2,2)
for i,filename in enumerate(all_files):
    image = skimage.io.imread(filename)
    ax.flatten()[i].imshow(image)
    #ax.flatten()[i].axis('off')
plt.show()

In [ ]:
def measure_root_mass(filename, sigma):

    # read the original image, converting to grayscale on the fly
    image = skimage.io.imread(fname=filename, as_gray=True)
    
    # blur before thresholding
    blurred_image = skimage.filters.gaussian(image, sigma=sigma)
    
    # perform adaptive thresholding to produce a binary image
    t = skimage.filters.threshold_otsu(blurred_image)
    binary_mask = blurred_image > t
    
    # determine root mass ratio
    rootPixels = np.count_nonzero(binary_mask)
    w = binary_mask.shape[1]
    h = binary_mask.shape[0]
    density = rootPixels / (w * h)

    return density

In [ ]:
measure_root_mass("trial-016.jpg", sigma=1.5)

In [ ]:
import glob
all_files = glob.glob("trial-???.jpg")
for filename in all_files:
    density = measure_root_mass(filename, sigma=1.5)
    # output in format suitable for .csv
    print(filename, density, sep=",")

## Ignoring more of the images -- implementation

In [ ]:
fig, ax = plt.subplots(2,2)
for i,filename in enumerate(all_files):
    image = skimage.io.imread(fname=filename, as_gray=True)
    blurred_image = skimage.filters.gaussian(image, sigma=1.5)
    t = skimage.filters.threshold_otsu(blurred_image)
    binary_mask = blurred_image > t
    ax.flatten()[i].imshow(binary_mask, cmap='gray')
    #ax.flatten()[i].axis('off')
plt.show()

In [ ]:
def enhanced_root_mass(filename, sigma):

    # read the original image, converting to grayscale on the fly
    image = skimage.io.imread(fname=filename, as_gray=True)
    
    # blur before thresholding
    blurred_image = skimage.filters.gaussian(image, sigma=sigma)
    
    # perform inverse binary thresholding to mask the white label and circle
    binary_mask = blurred_image > 0.95
    # use the mask to remove the circle and label from the blurred image
    blurred_image[binary_mask] = 0
    
    # perform adaptive thresholding to produce a binary image
    t = skimage.filters.threshold_otsu(blurred_image)
    binary_mask = blurred_image > t
    
    # determine root mass ratio
    rootPixels = np.count_nonzero(binary_mask)
    w = binary_mask.shape[1]
    h = binary_mask.shape[0]
    density = rootPixels / (w * h)

    return density

In [ ]:
all_files = glob.glob("trial-???.jpg")
for filename in all_files:
    density = enhanced_root_mass(filename, sigma=1.5)
    # output in format suitable for .csv
    print(filename, density, sep=",")

## Thresholding a bacteria colony image

In [ ]:
image = skimage.io.imread("../10-challenges/morphometrics/colonies01.tif")
gray_image = skimage.color.rgb2gray(image)
blurred_image = skimage.filters.gaussian(gray_image, sigma=1.0)
histogram, bin_edges = np.histogram(blurred_image, bins=256, range=(0.0, 1.0))

fig, ax = plt.subplots()
plt.plot(bin_edges[0:-1], histogram)
plt.title("Graylevel histogram")
plt.xlabel("gray value")
plt.ylabel("pixel count")
plt.xlim(0, 1.0)
plt.show()

In [ ]:
t = 0.2
binary_mask = blurred_image < t

fig, ax = plt.subplots()
plt.imshow(binary_mask, cmap='gray')
plt.show()